In [1]:
import pandas as pd
import requests
import json
import openai
from langchain_openai import ChatOpenAI
import os
from CredentialTo.CredentialToBrokerAPI import CredentialUpstox
import settings
from datetime import datetime
import upstox_client 
from tabulate import tabulate




In [2]:
df_futureOptions = pd.DataFrame()
masterData = pd.read_json("https://assets.upstox.com/market-quote/instruments/exchange/NSE.json.gz")
# masterData = requests.get(url).json() 
df_futureOptions = pd.DataFrame.from_dict(masterData)

In [3]:
df_futureOptions.head()

,weekly,segment,name,exchange,expiry,instrument_type,asset_symbol,underlying_symbol,instrument_key,lot_size,...,trading_symbol,strike_price,qty_multiplier,isin,security_type,short_name,asset_key,underlying_key,last_trading_date,price_quote_unit
0,0.0,NCD_FO,JPYINR,NSE,1.774636e+12,CE,JPYINR,JPYINR,NCD_FO|14294,1.0,...,JPYINR 61 CE 27 MAR 26,61.00,1000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,NCD_FO,JPYINR,NSE,1.774636e+12,PE,JPYINR,JPYINR,NCD_FO|14295,1.0,...,JPYINR 61 PE 27 MAR 26,61.00,1000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NSE_EQ,SDL RJ 7.49% 2035,NSE,NaN,SG,NaN,NaN,NSE_EQ|IN2920250163,100.0,...,749RJ35,NaN,1.0,IN2920250163,NORMAL,NaN,NaN,NaN,NaN,NaN
3,NaN,NSE_EQ,SDL RJ 7.57% 2043,NSE,NaN,SG,NaN,NaN,NSE_EQ|IN2920250171,100.0,...,757RJ43,NaN,1.0,IN2920250171,NORMAL,NaN,NaN,NaN,NaN,NaN
4,0.0,NCD_FO,GBPINR,NSE,1.767033e+12,PE,GBPINR,GBPINR,NCD_FO|14277,1.0,...,GBPINR 118.25 PE 29 DEC 25,118.25,1000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Convert Expiry to YYYY-MM-DD format

In [4]:
# Convert expiry from epoch (ms) to datetime
df_futureOptions["expiry"] = pd.to_datetime(df_futureOptions["expiry"], unit="ms")

# Convert datetime to string format YYYY-MM-DD
df_futureOptions["expiry"] = df_futureOptions["expiry"].dt.strftime("%Y-%m-%d")


# print(df_futureOptions.dtypes)  # verify expiry is now string
df_futureOptions.head()

,weekly,segment,name,exchange,expiry,instrument_type,asset_symbol,underlying_symbol,instrument_key,lot_size,...,trading_symbol,strike_price,qty_multiplier,isin,security_type,short_name,asset_key,underlying_key,last_trading_date,price_quote_unit
0,0.0,NCD_FO,JPYINR,NSE,2026-03-27,CE,JPYINR,JPYINR,NCD_FO|14294,1.0,...,JPYINR 61 CE 27 MAR 26,61.00,1000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,NCD_FO,JPYINR,NSE,2026-03-27,PE,JPYINR,JPYINR,NCD_FO|14295,1.0,...,JPYINR 61 PE 27 MAR 26,61.00,1000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NSE_EQ,SDL RJ 7.49% 2035,NSE,NaN,SG,NaN,NaN,NSE_EQ|IN2920250163,100.0,...,749RJ35,NaN,1.0,IN2920250163,NORMAL,NaN,NaN,NaN,NaN,NaN
3,NaN,NSE_EQ,SDL RJ 7.57% 2043,NSE,NaN,SG,NaN,NaN,NSE_EQ|IN2920250171,100.0,...,757RJ43,NaN,1.0,IN2920250171,NORMAL,NaN,NaN,NaN,NaN,NaN
4,0.0,NCD_FO,GBPINR,NSE,2025-12-29,PE,GBPINR,GBPINR,NCD_FO|14277,1.0,...,GBPINR 118.25 PE 29 DEC 25,118.25,1000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Fetch NEarest Expiry date from Today

In [5]:


# Today as Timestamp
today = pd.to_datetime(datetime.today().date())

# Convert expiry_dates to datetime
expiry_dates = pd.to_datetime(df_futureOptions["expiry"], errors="coerce")

# Filter only future expiries
future_expiries = expiry_dates[expiry_dates >= today]

# Find nearest expiry
nearest_expiry = future_expiries.min()

print("Nearest expiry:", nearest_expiry)


Nearest expiry: 2025-09-12 00:00:00


In [6]:
settings.expiry_list_upstox[0]

'2025-09-16'

Filter df_futureOptions BY : ["name"] == "NIFTY" | ["segment"] == "NSE_FO" | expiry

In [7]:
df_futureOptions = df_futureOptions[ (df_futureOptions["name"] == "NIFTY") & (df_futureOptions["segment"] == "NSE_FO")  & (df_futureOptions["expiry"] == settings.expiry_list_upstox[0]) ]
df_futureOptions =  df_futureOptions.reset_index(drop=True)
df_futureOptions

,weekly,segment,name,exchange,expiry,instrument_type,asset_symbol,underlying_symbol,instrument_key,lot_size,...,trading_symbol,strike_price,qty_multiplier,isin,security_type,short_name,asset_key,underlying_key,last_trading_date,price_quote_unit
0,1.0,NSE_FO,NIFTY,NSE,2025-09-16,CE,NIFTY,NIFTY,NSE_FO|44688,75.0,...,NIFTY 25050 CE 16 SEP 25,25050.0,1.0,NaN,NaN,NaN,NSE_INDEX|Nifty 50,NSE_INDEX|Nifty 50,NaN,NaN
1,1.0,NSE_FO,NIFTY,NSE,2025-09-16,PE,NIFTY,NIFTY,NSE_FO|44687,75.0,...,NIFTY 25000 PE 16 SEP 25,25000.0,1.0,NaN,NaN,NaN,NSE_INDEX|Nifty 50,NSE_INDEX|Nifty 50,NaN,NaN
2,1.0,NSE_FO,NIFTY,NSE,2025-09-16,CE,NIFTY,NIFTY,NSE_FO|44662,75.0,...,NIFTY 24950 CE 16 SEP 25,24950.0,1.0,NaN,NaN,NaN,NSE_INDEX|Nifty 50,NSE_INDEX|Nifty 50,NaN,NaN
3,1.0,NSE_FO,NIFTY,NSE,2025-09-16,PE,NIFTY,NIFTY,NSE_FO|44661,75.0,...,NIFTY 24900 PE 16 SEP 25,24900.0,1.0,NaN,NaN,NaN,NSE_INDEX|Nifty 50,NSE_INDEX|Nifty 50,NaN,NaN
4,1.0,NSE_FO,NIFTY,NSE,2025-09-16,CE,NIFTY,NIFTY,NSE_FO|44652,75.0,...,NIFTY 24900 CE 16 SEP 25,24900.0,1.0,NaN,NaN,NaN,NSE_INDEX|Nifty 50,NSE_INDEX|Nifty 50,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,1.0,NSE_FO,NIFTY,NSE,2025-09-16,PE,NIFTY,NIFTY,NSE_FO|44833,75.0,...,NIFTY 25600 PE 16 SEP 25,25600.0,1.0,NaN,NaN,NaN,NSE_INDEX|Nifty 50,NSE_INDEX|Nifty 50,NaN,NaN
164,1.0,NSE_FO,NIFTY,NSE,2025-09-16,PE,NIFTY,NIFTY,NSE_FO|44837,75.0,...,NIFTY 25700 PE 16 SEP 25,25700.0,1.0,NaN,NaN,NaN,NSE_INDEX|Nifty 50,NSE_INDEX|Nifty 50,NaN,NaN
165,1.0,NSE_FO,NIFTY,NSE,2025-09-16,PE,NIFTY,NIFTY,NSE_FO|44829,75.0,...,NIFTY 25500 PE 16 SEP 25,25500.0,1.0,NaN,NaN,NaN,NSE_INDEX|Nifty 50,NSE_INDEX|Nifty 50,NaN,NaN
166,1.0,NSE_FO,NIFTY,NSE,2025-09-16,CE,NIFTY,NIFTY,NSE_FO|44828,75.0,...,NIFTY 25500 CE 16 SEP 25,25500.0,1.0,NaN,NaN,NaN,NSE_INDEX|Nifty 50,NSE_INDEX|Nifty 50,NaN,NaN


Fetch NIFTY 50 instrument DATA

In [8]:
# Filter NIFTY 50 data
df_nifty50 = df_futureOptions[
    (df_futureOptions["instrument_key"] == "NSE_INDEX|Nifty 50")]

df_nifty50

,weekly,segment,name,exchange,expiry,instrument_type,asset_symbol,underlying_symbol,instrument_key,lot_size,...,trading_symbol,strike_price,qty_multiplier,isin,security_type,short_name,asset_key,underlying_key,last_trading_date,price_quote_unit


Fetch NIFTY LTP 

In [9]:
instrument_key='NSE_INDEX|Nifty 50'

df_ltp = pd.DataFrame



from upstox_client.rest import ApiException
from pprint import pprint

# Configure OAuth2 access token for authorization: OAUTH2
configuration = upstox_client.Configuration()
configuration.access_token = CredentialUpstox.ACCESS_TOKEN  
# create an instance of the API class
api_instance = upstox_client.MarketQuoteV3Api(upstox_client.ApiClient(configuration))

try:
    # Market quotes and instruments - LTP quotes.
    api_response = api_instance.get_ltp(instrument_key=instrument_key)
    
    #  Convert API response to dict
    data_dict = api_response.to_dict()
    
      # Convert to DataFrame
    df_ltp = df_ltp.from_dict(data_dict["data"], orient="index").reset_index()
    df_ltp.rename(columns={"index": "instrument_key"}, inplace=True)
    print(df_ltp)
    
    # pprint(api_response)
except ApiException as e:
    print("Exception when calling MarketQuoteV3Api->get_ltp: %s\n" % e)

       instrument_key  last_price    instrument_token  ltq  volume       cp
0  NSE_INDEX:Nifty 50    25024.25  NSE_INDEX|Nifty 50    0       0  24973.1


Filter By Strike Price

In [10]:

# Extract LTP from response dict
nifty_ltp_value = float(df_ltp["last_price"])

import math

# nifty50_ltp_data = 24740.6

# Round to nearest 50
nearest_strike = round(nifty_ltp_value / 50) * 50  

# Generate 5 strikes around LTP (2 below, 2 above, + the nearest one)
strike_list = [nearest_strike + i * 50 for i in range(-3, 3)]

# Filter DataFrame
df_futureOptions = df_futureOptions[
    df_futureOptions["strike_price"].isin(strike_list)
].reset_index(drop=True)

print("LTP:", nifty_ltp_value)
print("Nearest 5 strikes:", strike_list)


LTP: 25024.25
Nearest 5 strikes: [24850, 24900, 24950, 25000, 25050, 25100]


/var/folders/bm/2sqnclys19zb2dh55g0cx4zc0000gn/T/ipykernel_43547/701431457.py:2: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  nifty_ltp_value = float(df_ltp["last_price"])


FINAL MASTERDATA

In [11]:
df_futureOptions.head(11)


,weekly,segment,name,exchange,expiry,instrument_type,asset_symbol,underlying_symbol,instrument_key,lot_size,...,trading_symbol,strike_price,qty_multiplier,isin,security_type,short_name,asset_key,underlying_key,last_trading_date,price_quote_unit
0,1.0,NSE_FO,NIFTY,NSE,2025-09-16,CE,NIFTY,NIFTY,NSE_FO|44688,75.0,...,NIFTY 25050 CE 16 SEP 25,25050.0,1.0,NaN,NaN,NaN,NSE_INDEX|Nifty 50,NSE_INDEX|Nifty 50,NaN,NaN
1,1.0,NSE_FO,NIFTY,NSE,2025-09-16,PE,NIFTY,NIFTY,NSE_FO|44687,75.0,...,NIFTY 25000 PE 16 SEP 25,25000.0,1.0,NaN,NaN,NaN,NSE_INDEX|Nifty 50,NSE_INDEX|Nifty 50,NaN,NaN
2,1.0,NSE_FO,NIFTY,NSE,2025-09-16,CE,NIFTY,NIFTY,NSE_FO|44662,75.0,...,NIFTY 24950 CE 16 SEP 25,24950.0,1.0,NaN,NaN,NaN,NSE_INDEX|Nifty 50,NSE_INDEX|Nifty 50,NaN,NaN
3,1.0,NSE_FO,NIFTY,NSE,2025-09-16,PE,NIFTY,NIFTY,NSE_FO|44661,75.0,...,NIFTY 24900 PE 16 SEP 25,24900.0,1.0,NaN,NaN,NaN,NSE_INDEX|Nifty 50,NSE_INDEX|Nifty 50,NaN,NaN
4,1.0,NSE_FO,NIFTY,NSE,2025-09-16,CE,NIFTY,NIFTY,NSE_FO|44652,75.0,...,NIFTY 24900 CE 16 SEP 25,24900.0,1.0,NaN,NaN,NaN,NSE_INDEX|Nifty 50,NSE_INDEX|Nifty 50,NaN,NaN
5,1.0,NSE_FO,NIFTY,NSE,2025-09-16,PE,NIFTY,NIFTY,NSE_FO|44651,75.0,...,NIFTY 24850 PE 16 SEP 25,24850.0,1.0,NaN,NaN,NaN,NSE_INDEX|Nifty 50,NSE_INDEX|Nifty 50,NaN,NaN
6,1.0,NSE_FO,NIFTY,NSE,2025-09-16,PE,NIFTY,NIFTY,NSE_FO|44667,75.0,...,NIFTY 24950 PE 16 SEP 25,24950.0,1.0,NaN,NaN,NaN,NSE_INDEX|Nifty 50,NSE_INDEX|Nifty 50,NaN,NaN
7,1.0,NSE_FO,NIFTY,NSE,2025-09-16,CE,NIFTY,NIFTY,NSE_FO|44668,75.0,...,NIFTY 25000 CE 16 SEP 25,25000.0,1.0,NaN,NaN,NaN,NSE_INDEX|Nifty 50,NSE_INDEX|Nifty 50,NaN,NaN
8,1.0,NSE_FO,NIFTY,NSE,2025-09-16,PE,NIFTY,NIFTY,NSE_FO|44727,75.0,...,NIFTY 25100 PE 16 SEP 25,25100.0,1.0,NaN,NaN,NaN,NSE_INDEX|Nifty 50,NSE_INDEX|Nifty 50,NaN,NaN
9,1.0,NSE_FO,NIFTY,NSE,2025-09-16,CE,NIFTY,NIFTY,NSE_FO|44713,75.0,...,NIFTY 25100 CE 16 SEP 25,25100.0,1.0,NaN,NaN,NaN,NSE_INDEX|Nifty 50,NSE_INDEX|Nifty 50,NaN,NaN


In [12]:
df_futureOptions.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   weekly             12 non-null     float64
 1   segment            12 non-null     object 
 2   name               12 non-null     object 
 3   exchange           12 non-null     object 
 4   expiry             12 non-null     object 
 5   instrument_type    12 non-null     object 
 6   asset_symbol       12 non-null     object 
 7   underlying_symbol  12 non-null     object 
 8   instrument_key     12 non-null     object 
 9   lot_size           12 non-null     float64
 10  freeze_quantity    12 non-null     float64
 11  exchange_token     12 non-null     int64  
 12  minimum_lot        12 non-null     float64
 13  tick_size          12 non-null     float64
 14  asset_type         12 non-null     object 
 15  underlying_type    12 non-null     object 
 16  trading_symbol     12 non-nu

CONNECT TO WEBSOCKET LIVE FEED

Fetch All Instrument Keys to subscribe

In [13]:
# ActivateMarketFeed
# token_list = [99926000] is a NIFTY 50 token to get ltp of NIFTY 50 index
token_list = list(set(df_futureOptions['instrument_key'].tolist()))
print(token_list)


['NSE_FO|44667', 'NSE_FO|44713', 'NSE_FO|44688', 'NSE_FO|44662', 'NSE_FO|44687', 'NSE_FO|44706', 'NSE_FO|44727', 'NSE_FO|44640', 'NSE_FO|44668', 'NSE_FO|44652', 'NSE_FO|44661', 'NSE_FO|44651']


In [14]:
df_futureOptions.columns.tolist()

['weekly',
 'segment',
 'name',
 'exchange',
 'expiry',
 'instrument_type',
 'asset_symbol',
 'underlying_symbol',
 'instrument_key',
 'lot_size',
 'freeze_quantity',
 'exchange_token',
 'minimum_lot',
 'tick_size',
 'asset_type',
 'underlying_type',
 'trading_symbol',
 'strike_price',
 'qty_multiplier',
 'isin',
 'security_type',
 'short_name',
 'asset_key',
 'underlying_key',
 'last_trading_date',
 'price_quote_unit']

Version Update : 9 sep v2 

In [15]:
import time, os, json, uuid, re, threading, queue
import pandas as pd
from typing import List, Dict, Any
from openai import OpenAI
import upstox_client
from upstox_client.rest import ApiException

# ---- Initialize ----
access_token = CredentialUpstox.ACCESS_TOKEN
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

feed_col = [
    'Token','Open','High','Low','Close','Ltp',
    'Vol','Oi','theta','delta','gamma','vega','rho','iv','vtt'
]
df_feed = pd.DataFrame(columns=feed_col)
df_feed_enriched = pd.DataFrame()

# =========================================================
# Exit Manager Config & State
# =========================================================
class ExitConfig:
    target_pct = 0.05     # 5% target
    stop_pct = 0.03       # 3% stop loss
    trailing_pct = 0.02   # 2% trailing SL
    max_holding_sec = 3600  # 1 hour max hold
    dry_run = True

open_positions: Dict[str, dict] = {}  # token → position dict
exit_queue = queue.Queue()
exit_running = False

# =========================================================
# Exit Manager Core
# =========================================================
def record_fill(order: dict):
    """Record executed order as open position."""
    token = order["token"]
    open_positions[token] = {
        "side": order["side"],
        "qty": int(order["qty"]),
        "entry_price": order.get("price") or order.get("Ltp") or 0,
        "timestamp": time.time(),
        "max_price": order.get("price") or order.get("Ltp") or 0
    }
    print(f"📌 Recorded position: {token} @ {open_positions[token]}")

def should_exit_position(pos: dict, ltp: float, cfg: ExitConfig) -> str:
    """Return exit reason or None."""
    if pos["qty"] <= 0 or ltp <= 0:
        return None
    entry = pos["entry_price"]
    pnl_pct = (ltp - entry) / entry if pos["side"] == "BUY" else (entry - ltp) / entry
    elapsed = time.time() - pos["timestamp"]

    # Target
    if pnl_pct >= cfg.target_pct:
        return "TARGET"
    # Stop loss
    if pnl_pct <= -cfg.stop_pct:
        return "STOPLOSS"
    # Trailing SL
    if pos["side"] == "BUY":
        pos["max_price"] = max(pos["max_price"], ltp)
        if ltp < pos["max_price"] * (1 - cfg.trailing_pct):
            return "TRAILING_SL"
    else:  # SELL side
        pos["max_price"] = min(pos["max_price"], ltp)
        if ltp > pos["max_price"] * (1 + cfg.trailing_pct):
            return "TRAILING_SL"
    # Time exit
    if elapsed > cfg.max_holding_sec:
        return "TIME_EXIT"
    return None

def place_exit_order_upstox(token: str, pos: dict, cfg: ExitConfig):
    """Send exit order (reverse side)."""
    side = "SELL" if pos["side"] == "BUY" else "BUY"
    qty = pos["qty"]
    if cfg.dry_run:
        print(f"💡 [DRY RUN] Exit {side} {qty} {token}")
        return {"status": "simulated", "side": side, "qty": qty, "token": token}
    try:
        configuration = upstox_client.Configuration()
        configuration.access_token = access_token
        api_client = upstox_client.ApiClient(configuration)
        order_api = upstox_client.OrderApi(api_client)

        order_req = upstox_client.PlaceOrderRequest(
            quantity=str(qty),
            product="I",  # Intraday
            validity="DAY",
            price=0.0,
            tag="EXIT-ENGINE",
            instrument_token=token,
            order_type="MARKET",
            transaction_type=side,
            disclosed_quantity=0,
            trigger_price=0,
            is_amo=False
        )
        resp = order_api.place_order(body=order_req, api_version="3.0")
        return {"status": "placed", "order_id": resp.data.order_id, "side": side, "qty": qty, "token": token}
    except ApiException as e:
        return {"status": "error", "reason": str(e)}

def enqueue_snapshot_for_exit(snapshot_df: pd.DataFrame):
    exit_queue.put(snapshot_df.copy())

def exit_worker(cfg: ExitConfig):
    global exit_running
    while exit_running:
        try:
            snapshot = exit_queue.get(timeout=1)
        except queue.Empty:
            continue
        for token, pos in list(open_positions.items()):
            if token not in snapshot["Token"].values:
                continue
            ltp = snapshot.loc[snapshot["Token"] == token, "Ltp"].iloc[0]
            reason = should_exit_position(pos, ltp, cfg)
            if reason:
                print(f"⚡ Exiting {token} due to {reason}")
                res = place_exit_order_upstox(token, pos, cfg)
                print("➡️ Exit Order:", res)
                open_positions.pop(token, None)

def start_exit_manager(cfg: ExitConfig):
    global exit_running
    exit_running = True
    t = threading.Thread(target=exit_worker, args=(cfg,), daemon=True)
    t.start()
    print("✅ Exit manager started")

def stop_exit_manager():
    global exit_running
    exit_running = False
    print("🛑 Exit manager stopped")



Tasks  - 


⚠️ Notes & Limitations

With only ₹3k, buying is the only feasible strategy (no spreads/hedges).

Requires clear downside move for large % gains.

Must monitor OI, bid-ask spreads, and underlying NIFTY closely.

Always use limit orders for better fills.


--  Add  monitor OI, bid-ask spreads, and underlying NIFTY to df_feed and into LLM Prompt also


IDEAS :

-   currently I'm calling LLM every 10s ... instead of this I can say Analyse the LIVE feed data for 15 mins ( or specific time) then create strategy and execute order . 

In [16]:
import time
import os, json
import pandas as pd
from openai import OpenAI
import upstox_client
import re, json
import threading



# ---- Initialize ----
access_token = CredentialUpstox.ACCESS_TOKEN
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

feed_col = [
    'Token','Open', 'High', 'Low', 'Close', 'Ltp',
    'Vol', 'Oi', 'theta', 'delta', 'gamma', 'vega', 'rho', 'iv', 'vtt'
]
df_feed = pd.DataFrame(columns=feed_col)
df_feed_enriched = pd.DataFrame()   # ✅ initialize so it's always accessible



import os, json, time, uuid
import pandas as pd
from typing import List, Dict, Any




# =========================================================
# 1. Merge df_futureOptions columns into df_feed
# =========================================================
def enrich_feed(df_feed: pd.DataFrame, df_futureOptions: pd.DataFrame, 
                cols_to_add: List[str] = None) -> pd.DataFrame:
    """
    Merge option metadata into df_feed.
    Columns added: lot_size, trading_symbol, strike_price, tick_size, instrument_type
    """
    if cols_to_add is None:
        cols_to_add = ['lot_size','trading_symbol','strike_price','tick_size','instrument_type']
    
    df_feed = df_feed.copy()
    df_futureOptions = df_futureOptions.copy()
    
    if 'Token' not in df_feed.columns:
        raise KeyError("df_feed must contain 'Token'")
    if 'instrument_key' not in df_futureOptions.columns:
        raise KeyError("df_futureOptions must contain 'instrument_key'")
    
    # Normalize keys
    df_feed['Token'] = df_feed['Token'].astype(str).str.strip()
    df_futureOptions['instrument_key'] = df_futureOptions['instrument_key'].astype(str).str.strip()
    
    # Deduplicate
    df_futureOptions = df_futureOptions.drop_duplicates(subset=['instrument_key'], keep='last')
    
    # Only keep available cols
    available = [c for c in cols_to_add if c in df_futureOptions.columns]
    if not available:
        print("[enrich_feed] Warning: no metadata columns found to merge.")
        return df_feed
    
    merged = df_feed.merge(
        df_futureOptions[['instrument_key'] + available],
        left_on='Token',
        right_on='instrument_key',
        how='left',
        validate='m:1'
    )
    
    if 'instrument_key' in merged.columns:
        merged = merged.drop(columns=['instrument_key'])
    
    # Convert dtypes
    if 'lot_size' in available:
        merged['lot_size'] = pd.to_numeric(merged['lot_size'], errors='coerce').astype('Int64')
    if 'strike_price' in available:
        merged['strike_price'] = pd.to_numeric(merged['strike_price'], errors='coerce')
    
    return merged


# =========================================================
# 2. Build LLM prompt
# =========================================================
def build_prompt_from_feed(snapshot_df: pd.DataFrame, max_rows: int = 8) -> str:
    """
    Turn latest snapshot rows into a textual prompt for LLM.
    """
    rows = snapshot_df.tail(max_rows).to_dict(orient='records')
    lines = []
    for r in rows:
        lines.append(
            f"Token:{r.get('Token')} sym:{r.get('trading_symbol')} type:{r.get('instrument_type')} "
            f"strike:{r.get('strike_price')} LTP:{r.get('Ltp')} IV:{r.get('iv')} OI:{r.get('Oi')} "
            f"Δ={r.get('delta')} Γ={r.get('gamma')} Θ={r.get('theta')} Vega={r.get('vega')} "
            f"lot:{r.get('lot_size')} tick:{r.get('tick_size')}"
        )
    feed_text = "\n".join(lines)
    
    prompt = f"""You are an expert Nifty 50 options strategist. Your objective is to analyze the price and volume patterns of NIFTY 50 option chain
    live data coming from websocket. identify potential buying opportunities. Utilize advanced charting tools and technical 
    indicators to scrutinize both short-term and long-term patterns, taking into account historical data and recent market movements. 
    I want intraday buying strategies with less than 3000 rupees amount and provide high profit and very low risks . Use Greeks data to deep analysis of options.

    Given the following live feed snapshot:

    {feed_text}

    Constraints:
    - Provide 1–3 legs only.
    - Total amount must not exceed 3000 rupees. 
    - Use Greeks data for advanced analysis.
    - I want buying strategies only with high profit and low risk.
    - Each leg must specify token, side (BUY/SELL), qty, order_type, price, product.
    - Qty must not exceed 300 and should be a multiple of lot_size.
    - Respond ONLY in JSON with keys: strategy_id, rationale, legs.
    """
    return prompt


# =========================================================
# 3. LLM call (mock or real)
# =========================================================
def ask_llm_for_strategy(snapshot_df: pd.DataFrame, use_mock=False) -> Dict[str,Any]:
    prompt = build_prompt_from_feed(snapshot_df)
    print("=== PROMPT SENT TO LLM ===\n", prompt)
    
    if use_mock:
        # Simple mock strategy for testing
        rows = snapshot_df.tail(2).to_dict(orient='records')
        legs = []
        for r in rows:
            legs.append({
                "token": r['Token'],
                "side": "SELL" if (r.get('theta', 0) < -10) else "BUY",
                "qty": int(min(300, (r.get('lot_size') or 1) * 1)),  # one lot
                "order_type": "MARKET",
                "price": None,
                "product": "NRML"
            })
        return {
            "strategy_id": str(uuid.uuid4()),
            "rationale": "Mock strategy: short theta if theta is very negative.",
            "legs": legs
        }
    else:
        # Real OpenAI call
        client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
        resp = client.chat.completions.create(
            model="gpt-5-mini",
            temperature=1,
            messages=[
                {"role": "system", "content": "You are an expert trading assistant for Nifty options. You have deep knowledge of options trading strategies and human psychology."},
                {"role": "user", "content": prompt}
            ]
        )
        raw = resp.choices[0].message.content
        
        # Extract JSON substring if necessary
        import re
        match = re.search(r'(\{.*\})', raw, flags=re.S)
        if match:
            return json.loads(match.group(1))
        return json.loads(raw)


# =========================================================
# 4. Validate LLM response
# =========================================================
def validate_strategy(plan: Dict[str,Any], df_feed: pd.DataFrame) -> Dict[str,Any]:
    """
    Validate schema and enforce safety rules.
    """
    if not isinstance(plan, dict): 
        raise ValueError("Plan must be a dict")
    if 'legs' not in plan or not isinstance(plan['legs'], list):
        raise ValueError("Plan missing 'legs' list")
    
    for leg in plan['legs']:
        if not all(k in leg for k in ['token','side','qty']):
            raise ValueError(f"Leg missing keys: {leg}")
        if leg['token'] not in df_feed['Token'].values:
            raise ValueError(f"Unknown token: {leg['token']}")
        if int(leg['qty']) > 300:
            raise ValueError(f"Qty exceeds 300: {leg['qty']}")
        lot = df_feed.loc[df_feed['Token']==leg['token'],'lot_size'].iloc[0]
        if pd.notna(lot) and int(leg['qty']) % int(lot) != 0:
            raise ValueError(f"Qty {leg['qty']} not multiple of lot size {lot}")
    
    return plan


# =========================================================
# 5. Place Orders (dry run)
# =========================================================
from upstox_client.rest import ApiException

def place_orders(plan: Dict[str,Any], df_feed: pd.DataFrame, dry_run=True) -> List[Dict[str,Any]]:
    results = []

    configuration = upstox_client.Configuration()
    configuration.access_token = CredentialUpstox.ACCESS_TOKEN
    api_client = upstox_client.ApiClient(configuration)
    order_api = upstox_client.OrderApi(api_client)

    # ✅ Mapping for product
    product_map = {
        "NRML": "D",   # Delivery (normal)
        "MIS": "I",    # Intraday
        "CO": "CO",
        "OCO": "OCO",
        "MTF": "MTF"
    }

    for leg in plan['legs']:
        token = leg['token']
        qty = int(leg['qty'])
        side = leg['side'].upper()
        order_type = leg.get('order_type', 'MARKET').upper()
        product = leg.get('product', 'NRML').upper()
        price = leg.get('price') or 0.0

        # Map to Upstox enum
        product_code = product_map.get(product, "I")  # default to Delivery

        row = df_feed.loc[df_feed['Token'] == token].iloc[0]
        trading_symbol = row.get('trading_symbol')

        if qty > 300:
            results.append({"status": "rejected", "reason": "qty>300", "leg": leg})
            continue

        lot = int(row.get('lot_size')) if pd.notna(row.get('lot_size')) else None
        if lot and (qty % lot != 0):
            results.append({"status": "rejected", "reason": "qty_not_multiple_of_lot", "leg": leg})
            continue

        if dry_run:
            results.append({
                "status": "simulated",
                "token": token,
                "trading_symbol": trading_symbol,
                "qty": qty,
                "side": side,
                "order_type": order_type,
                "product": product_code
            })
            continue
        try:
            order_req = upstox_client.PlaceOrderRequest(
                quantity="75", # quantity should be string and multiple of lot size (e.g., 75 for NIFTY)
                product=product_code,
                validity="DAY",
                price=price,
                tag="LLM-STRATEGY",
                instrument_token=token,
                order_type=order_type,
                transaction_type=side,
                disclosed_quantity=0,
                trigger_price=0,
                is_amo=False
            )

            # 🔹 FIX: Add api_version
            resp = order_api.place_order(body=order_req, api_version="3.0")

            results.append({
                "status": "placed",
                "order_id": resp.data.order_id,
                "token": token,
                "trading_symbol": trading_symbol,
                "qty": qty,
                "side": side
            })
        except ApiException as e:
            results.append({"status": "error", "reason": str(e), "leg": leg})


    return results

# =========================================================
# Example Usage
# =========================================================
# Suppose df_feed and df_futureOptions are already defined in your notebook:
# df_feed = ...   # live feed DataFrame from websocket
# df_futureOptions = ...   # instrument metadata DataFrame

# 1. Enrich feed
# df_enriched = enrich_feed(df_feed, df_futureOptions)

# 2. Ask LLM
# plan = ask_llm_for_strategy(df_enriched, use_mock=True)   # use_mock=False for real call

# 3. Validate
# validated = validate_strategy(plan, df_enriched)

# 4. Place orders (dry run)
# orders = place_orders(validated, df_enriched, dry_run=True)
# print(orders)


# ---- WebSocket Handlers ----
_last_eval = 0  # global timer

def on_message(message):
    global df_feed, df_futureOptions, df_feed_enriched, _last_eval

    # Parse incoming message
    try:
        if isinstance(message, str):
            feeds = json.loads(message).get("feeds", {})
        elif isinstance(message, dict):
            feeds = message.get("feeds", {})
        else:
            print("Unexpected message type:", type(message))
            return
    except Exception as e:
        print("Failed to parse message:", e)
        return

    # Update df_feed quickly
    for token, feed_data in feeds.items():
        marketFF = feed_data.get("fullFeed", {}).get("marketFF", {})
        ltpc = marketFF.get("ltpc", {})
        ohlc_list = marketFF.get("marketOHLC", {}).get("ohlc", [])
        greeks = marketFF.get("optionGreeks", {})

        # Daily OHLC snapshot
        ohlc = next((item for item in ohlc_list if item.get("interval") == "1d"), {})

        row = {
            "Token": token,
            "Open": float(ohlc.get("open", 0)),
            "High": float(ohlc.get("high", 0)),
            "Low": float(ohlc.get("low", 0)),
            "Close": float(ohlc.get("close", 0)),
            "Ltp": float(ltpc.get("ltp", 0)),
            "Vol": float(ohlc.get("vol", 0)),
            "Oi": float(marketFF.get("oi", 0)),
            "theta": float(greeks.get("theta", 0)),
            "delta": float(greeks.get("delta", 0)),
            "gamma": float(greeks.get("gamma", 0)),
            "vega": float(greeks.get("vega", 0)),
            "rho": float(greeks.get("rho", 0)),
            "iv": float(marketFF.get("iv", 0)),
            "vtt": float(marketFF.get("vtt", 0)),
        }

        if token in df_feed["Token"].values:
            df_feed.loc[df_feed["Token"] == token, list(row.keys())] = list(row.values())
        else:
            df_feed = pd.concat([df_feed, pd.DataFrame([row])], ignore_index=True)

    # Enrich with df_futureOptions metadata (kept global for viewing in new cells)
    df_feed_enriched = enrich_feed(df_feed, df_futureOptions)

    # Every 10s: build strategy -> place orders -> trigger exit manager
    if time.monotonic() - _last_eval > 10 and not df_feed_enriched.empty:
        try:
            snapshot_df = df_feed_enriched.tail(20)

            # Toggle mock/real and dry_run as needed
            plan = ask_llm_for_strategy(snapshot_df, use_mock=False)            # True for testing
            validated = validate_strategy(plan, df_feed_enriched)
            orders = place_orders(validated, df_feed_enriched, dry_run=False)   # True for testing
            print("\n✅ Strategy:", json.dumps(plan, indent=2))

            # Pretty show orders (Jupyter)
            try:
                from IPython.display import display
                if orders:
                    display(pd.DataFrame(orders))
                else:
                    print("📦 No orders generated")
            except Exception:
                print("📦 Orders:", orders)

            # 🔔 Trigger exit manager snapshot when orders were placed (or simulated if testing)
            placed_any = any(o.get("status") in ("placed", "simulated") for o in orders)
            if placed_any:
                enqueue_snapshot_for_exit(df_feed_enriched.copy())

        except Exception as e:
            print("⚠️ Strategy generation failed:", e)

        _last_eval = time.monotonic()

    # Debug: show recent feed rows
    df_feed_enriched.head(11)



def on_error(error):
    print("Error:", error)


def on_close():
    print("WebSocket closed")


# ---- Connect to Upstox ----
configuration = upstox_client.Configuration()
configuration.access_token = access_token

instrument_key = token_list  # e.g., 'NSE_INDEX|Nifty 50'
mode = 'full'  # full_d30, ltpc, full, etc.

streamer = upstox_client.MarketDataStreamerV3(
    upstox_client.ApiClient(configuration),
    instrument_key,
    mode=mode
)

# Start exit manager (safe defaults; set .dry_run=False when you are confident)
cfg = ExitConfig()
cfg.dry_run = False
start_exit_manager(cfg)


streamer.on("message", on_message)
streamer.on("error", on_error)
streamer.on("close", on_close)

streamer.connect()

while True:
    time.sleep(1)   # Keep main thread alive to maintain WebSocket connection




✅ Exit manager started


/var/folders/bm/2sqnclys19zb2dh55g0cx4zc0000gn/T/ipykernel_43547/647170663.py:354: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_feed = pd.concat([df_feed, pd.DataFrame([row])], ignore_index=True)


=== PROMPT SENT TO LLM ===
 You are an expert Nifty 50 options strategist. Your objective is to analyze the price and volume patterns of NIFTY 50 option chain
    live data coming from websocket. identify potential buying opportunities. Utilize advanced charting tools and technical 
    indicators to scrutinize both short-term and long-term patterns, taking into account historical data and recent market movements. 
    I want intraday buying strategies with less than 3000 rupees amount and provide high profit and very low risks . Use Greeks data to deep analysis of options.

    Given the following live feed snapshot:

    Token:NSE_FO|44687 sym:NIFTY 25000 PE 16 SEP 25 type:PE strike:25000.0 LTP:68.15 IV:0.077056884765625 OI:21282525.0 Δ=-0.413 Γ=0.0017 Θ=-8.7596 Vega=11.4659 lot:75 tick:5.0
Token:NSE_FO|44668 sym:NIFTY 25000 CE 16 SEP 25 type:CE strike:25000.0 LTP:126.5 IV:0.08544921875 OI:14014350.0 Δ=0.5789 Γ=0.0016 Θ=-9.7557 Vega=11.5156 lot:75 tick:5.0
Token:NSE_FO|44688 sym:NIFT

In [17]:
df_feed_enriched.head(11)

""


,status,order_id,token,trading_symbol,qty,side
0,placed,250911000171438,NSE_FO|44661,NIFTY 24900 PE 16 SEP 25,75,BUY
